# ONT for bacterial SNP calling

* tools used: medaka, guppy

In [10]:
import sys,os,shutil,subprocess
import glob,time
from importlib import reload
import numpy as np
import pandas as pd

## test guppy command

In [27]:
def run_guppy(path,out,threads_per_caller,num_callers,chunks_per_runner,gpu_runners_per_device):
        
    cmd = 'guppy_basecaller -i {p} -s {o} --cpu_threads_per_caller {tpc} --num_callers {nc} --chunks_per_runner {cpr} \
        --gpu_runners_per_device {grpd} \
        -c {c} -x "cuda:0"'.format(o=out,p=path,c=cfg,tpc=threads_per_caller,nc=num_callers,
                                   cpr=chunks_per_runner,grpd=gpu_runners_per_device)
    print (cmd)
    st=time.time()
    tmp = subprocess.check_output(cmd, shell=True)
    elapsed=time.time()-st
    return elapsed

In [ ]:
cfg='/home/farrell/gitprojects/nanopore-project/test/custom.cfg' 
res=[]
for x in range(1,10,1):
    t = run_guppy('test/fast5','test/basecall_fast/',20,20,512,x)
    print (x,t)
    res.append([x,t])

In [ ]:
res=pd.DataFrame(res,columns=['metric','time'])
res.plot(x='metric')

In [17]:
cfg='/home/farrell/gitprojects/nanopore-project/test/custom.cfg'
run_guppy('test/fast5','test/basecall_fast'20,20,512,5)

guppy_basecaller -i test/fast5 -s test/basecall_fast --cpu_threads_per_caller 20 --num_callers 20 --chunks_per_runner 512         --gpu_runners_per_device 5         -c /home/farrell/gitprojects/nanopore-project/test/custom.cfg -x "cuda:0"


74.63200640678406

In [14]:
cfg='/home/farrell/gitprojects/nanopore-project/test/custom_hac.cfg'
run_guppy('test/fast5','test/basecall_hac',20,20,512,5)

guppy_basecaller -i test/fast5 -s test/basecall --cpu_threads_per_caller 20 --num_callers 20 --chunks_per_runner 512         --gpu_runners_per_device 5         -c /home/farrell/gitprojects/nanopore-project/test/custom_hac.cfg -x "cuda:0"


522.2056367397308

## bonito caller

## join fastqs

In [23]:
# join fastqs
def join_fastq(path, out):
    import gzip
    files = os.path.join(path,'*.fastq')
    cmd = 'cat {f} > {o}'.format(f=files,o=out)
    print (cmd)
    subprocess.check_output(cmd,shell=True)
    cmd = 'gzip %s' %out
    print ('compressing')
    subprocess.check_output(cmd,shell=True)
    return

join_fastq('test/basecall_hac/pass/', 'test/hac_joined.fastq')

cat test/basecall_hac/pass/*.fastq > hac_joined.fastq


## align with minimap2

In [34]:
out = 'test/hac.bam'
ref = 'phage_lambda.fasta'
#query = 'test/basecall_hac/pass/fastq_runid_15a87511c08725ce03507bdf86e85cdf36ec5a02_0_0.fastq'
query = 'test/hac_joined.fastq.gz'

def align_minimap2(query,out,ref,threads=4):
    cmd = 'minimap2 -ax map-ont -t {t} {r} {q} | samtools sort -o {o}'.format(o=out,r=ref,q=query,t=threads)
    print (cmd)
    subprocess.check_output(cmd,shell=True)
    
#align_minimap2(query,out,ref)

## m.bovis samples

In [ ]:
cfg='/home/farrell/gitprojects/nanopore-project/test/custom_hac.cfg'
path = '/storage/ONT/TB19-3825_05_OCT_21_GS/TB19-3825/20211005_1220_MN37978_FAQ44893_37c3a1fa/fast5/'
run_guppy(path,'test/TB19-3825',20,20,512,5)

In [42]:
name = 'TB20-3738'
join_fastq('test/%s/pass'% name, 'test/%s.fastq' %name)
ref = 'Mbovis_AF212297.fa'
align_minimap2('test/%s.fastq.gz' %name,'test/%s.bam' %name,ref,20)

cat test/TB20-3738/pass/*.fastq > test/TB20-3738.fastq
minimap2 -ax map-ont -t 20 Mbovis_AF212297.fa test/TB20-3738.fastq.gz | samtools sort -o test/TB20-3738.bam


## find depth/coverage of runs

## call variants with snipgenie

In [46]:
from snipgenie import app, tools

bam_files =app.get_files_from_paths('test/mapped_mbovis/', '*.bam')
gff_file='mbovis.gff'
tools.gff_bcftools_format(app.mbovis_gb, gff_file)
app.run_bamfiles(bam_files, app.mbovis_genome, outdir='test/variantcall', 
                 threads=20,
                 mask=app.mbovis_mask, gff_file=gff_file,
                 custom_filters=True, overwrite=False)

[      1  217496  434991  652486  869981 1087476 1304971 1522467 1739962
 1957457 2174952 2392447 2609942 2827437 3044933 3262428 3479923 3697418
 3914913 4132408 4349904]
parallel bcftools mpileup -r {1} -a "AD,ADF,ADR,DP,SP,INFO/AD,INFO/ADF,INFO/ADR" -O b --min-MQ 10 -o {2} -f /home/farrell/.config/snipgenie/genome/Mbovis_AF212297.fa test/mapped_mbovis/TB20-3762.bam test/mapped_mbovis/TB19-3825.bam test/mapped_mbovis/TB20-3738.bam ::: "LT708304.1":1-217495 "LT708304.1":217496-434990 "LT708304.1":434991-652485 "LT708304.1":652486-869980 "LT708304.1":869981-1087475 "LT708304.1":1087476-1304970 "LT708304.1":1304971-1522466 "LT708304.1":1522467-1739961 "LT708304.1":1739962-1957456 "LT708304.1":1957457-2174951 "LT708304.1":2174952-2392446 "LT708304.1":2392447-2609941 "LT708304.1":2609942-2827436 "LT708304.1":2827437-3044932 "LT708304.1":3044933-3262427 "LT708304.1":3262428-3479922 "LT708304.1":3479923-3697417 "LT708304.1":3697418-3914912 "LT708304.1":3914913-4132407 "LT708304.1":4132408-4